In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
import trimesh
import numpy as np

# Initialiser le dispositif (CPU ou GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Charger le modèle VAE
vae = VAE().to(device)
model_path = 'vae_models/vae_model.pth'

# Vérifiez que le fichier modèle existe
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Le fichier modèle n'existe pas : {model_path}")

# Charger les poids du modèle
vae.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))

vae.eval()
print("Modèle VAE chargé avec succès.")

# Transformation des images 2D en tenseur
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize((0.5,), (0.5,))
])

# Fonction pour sauvegarder le modèle 3D en utilisant Trimesh
def save_as_obj_with_trimesh(voxel_tensor, output_path):
    """
    Sauvegarde les voxels comme un mesh triangulé (fichier OBJ) en utilisant Trimesh.

    Args:
        voxel_tensor (torch.Tensor): Tenseur 3D (ou numpy array) contenant les voxels générés.
        output_path (str): Chemin pour sauvegarder le fichier OBJ.
    """
    # Convertir le tenseur en numpy array
    voxel_tensor = voxel_tensor.squeeze().cpu().numpy()

    # Normaliser les valeurs si nécessaire
    voxel_tensor = (voxel_tensor - voxel_tensor.min()) / (voxel_tensor.max() - voxel_tensor.min())

    # Seuillage pour convertir en voxels binaires
    voxel_tensor = (voxel_tensor > 0.1).astype(np.uint8)

    if voxel_tensor.sum() == 0:
        raise ValueError("Le volume ne contient pas de voxels activés après seuillage.")

    # Générer un mesh triangulé avec trimesh
    mesh = trimesh.voxel.ops.matrix_to_marching_cubes(voxel_tensor)

    # Debug : Afficher des informations sur le mesh généré
    print(f"Mesh info: vertices={len(mesh.vertices)}, faces={len(mesh.faces)}")

    if len(mesh.vertices) == 0 or len(mesh.faces) == 0:
        raise ValueError("Le mesh généré est vide. Vérifiez les données d'entrée.")

    # Sauvegarder le mesh au format OBJ
    mesh.export(output_path)
    print(f"Mesh sauvegardé au format OBJ : {output_path}")

# Fonction de génération du modèle 3D
def generate_3d_model(image_path, vae, output_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    vae.eval()
    with torch.no_grad():
        generated_voxel, _, _ = vae(image)
        print(f"Voxel Tensor Stats - Min: {generated_voxel.min()}, Max: {generated_voxel.max()}, Shape: {generated_voxel.shape}")
        save_as_obj_with_trimesh(generated_voxel, output_path)

# Exemple d'utilisation
# Exemple d'appel
generate_3d_model('pix3d/img/sofa/0103.png', vae, './generated_vae_model_sofa.obj')
print("Modèle 3D généré et sauvegardé.")


Modèle VAE chargé avec succès.
Voxel Tensor Stats - Min: 2.5153858587145805e-05, Max: 0.42821717262268066, Shape: torch.Size([1, 1, 32, 32, 32])
Mesh info: vertices=9230, faces=18496
Mesh sauvegardé au format OBJ : ./generated_vae_model_sofa.obj
Modèle 3D généré et sauvegardé.


In [ ]:
mesh = trimesh.load('./generated_vae_model_trimesh.obj')
print(mesh.is_empty)  # Devrait retourner False si le mesh contient des faces
mesh.show()  # Affiche le mesh dans une visualisation interactive


False
